# Train Diffusion Model
### Imports

In [1]:
try: 
    import librosa
except:
    !pip install librosa


#Set Dir 
import sys, os
sys.path.append(os.path.abspath('..'))

# Torch
import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader
import torch.optim as optim

# Utils
import numpy as np
from numpy import ndarray
import logging

# Base Scripts
from Libraries.U_Net import *
from Libraries.Diffusion import *
from Libraries.Utils import *
from MainScripts.Conf import conf

### Config
General

In [2]:
remote_kernel: bool = True

logging_level: int = LIGHT_DEBUG #logging.INFO
model_name: str = "diffusion_v19"
full_model_path: str = path_to_remote_path("{}/{}".format(conf["paths"].model_path, model_name + ".pth"), remote_kernel)
checkpoint_freq: int = 10 #0 for no checkpoint saving
training_data_name: str = "training_full_low_res"

device = "cuda" if torch.cuda.is_available() else "cpu"
restart_training: bool = True #If True and model already exists optimizer and lr_scheduler are reset
learning_rate: float = 5e-4 #Starting lr/first lr for Threshold Scheduler
epochs: int = 300
n_training_samples: int = 1280

logging.basicConfig(level=logging_level, format='%(asctime)s - %(levelname)s - %(message)s')
logger: logging.Logger = logging.getLogger(__name__)

### Data Loading

In [3]:
file: ndarray = load_training_data(path_to_remote_path("{}/{}".format(conf["paths"].data_path, training_data_name + ".npy"), remote_kernel))[:n_training_samples, ...]
data_loader = create_dataloader(Audio_Data(file), conf["model"].batch_size)
logger.info(f"Data loaded with shape: {file.shape}")

2025-03-25 09:51:26,258 - LIGHT_DEBUG - Ndarray loaded from Data/training_full_low_res.npy of shape: (7087, 224, 416)
2025-03-25 09:51:26,500 - INFO - Data loaded with shape: (1280, 224, 416)


### Model Creation
U-Net

In [4]:
u_net = Conv_U_NET(in_channels=1,
                    time_embed_dim=conf["model"].time_embed_dim, 
                    n_starting_filters=conf["model"].n_starting_filters, 
                    n_downsamples=conf["model"].n_downsamples, 
                    activation=nn.ELU(), 
                    device=device,
                    conditional_norm=False
                ).to(device)

optimizer = optim.AdamW(u_net.parameters(), lr=learning_rate)
#scheduler = Threshold_LR(optimizer, [1, 0.1, 0.09, 0.85, 0.08, 0.07], [learning_rate, 1e-4, 1e-5, 5e-6, 1e-6, 1e-7])
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
start_epoch: int = 0
if os.path.exists(full_model_path):
    model = torch.load(full_model_path, map_location=device)
    u_net.load_state_dict(model["model"])
    if not restart_training:
        optimizer.load_state_dict(model["optim"])
        scheduler.load_state_dict(model["scheduler"])
        start_epoch = model.get("epoch", 0)
    logger.info(f"Model {model_name} loaded with {count_parameters(u_net)} Parameters")
else: 
    logger.info(f"Model {model_name} created with {count_parameters(u_net)} Parameters")



2025-03-25 09:51:27,691 - INFO - Model diffusion_v19 created with 17288545 Parameters


Diffusion

In [5]:
diffusion = Diffusion(model=u_net, 
                        noise_steps=conf["model"].diffusion_timesteps, 
                        noise_schedule="cosine", 
                        input_dim=[conf["model"].batch_size, 1, file.shape[-2], file.shape[-1]],
                        device=device
                    )

#diffusion.visualize_diffusion_steps(x=Tensor(file[:100]), n_spectograms=10)

### Train

In [6]:
x = diffusion.train(epochs=epochs, 
                    data_loader=data_loader, 
                    loss_function=nn.MSELoss(),
                    optimizer=optimizer, 
                    lr_scheduler=scheduler, 
                    gradient_accum=conf["model"].gradient_accum,
                    checkpoint_freq=checkpoint_freq, 
                    model_path=full_model_path, 
                    start_epoch=start_epoch,
                    patience=100
                )
scatter_plot(x)

2025-03-25 09:51:27,980 - INFO - Training started on cuda


2025-03-25 09:53:21,000 - LIGHT_DEBUG - Batch 80/80 Pred noise min/max: -0.03679, 0.03354 std/mean: 0.31789, -0.00074 Real noise min/max: -0.07159, 0.07359 std/mean: 0.99997, 0.00019 


2025-03-25 09:53:21,800 - INFO - Epoch 001: Avg. Loss: 7.68800e-01 Remaining Time: 09h 27min 10s LR: 4.99986e-04 


2025-03-25 09:55:15,000 - LIGHT_DEBUG - Batch 80/80 Pred noise min/max: -0.04399, 0.04639 std/mean: 0.52764, -0.00022 Real noise min/max: -0.07351, 0.07044 std/mean: 0.99990, 0.00005 


2025-03-25 09:55:15,847 - INFO - Epoch 002: Avg. Loss: 4.94908e-01 Remaining Time: 09h 25min 50s LR: 4.99945e-04 


...

2025-03-25 11:14:26,872 - INFO - Epoch 044: Avg. Loss: 1.29943e-01 Remaining Time: 08h 02min 39s LR: 4.73928e-04 


2025-03-25 11:16:19,000 - LIGHT_DEBUG - Batch 80/80 Pred noise min/max: -0.07153, 0.07236 std/mean: 0.93045, -0.00050 Real noise min/max: -0.06968, 0.07437 std/mean: 1.00001, -0.00000 


2025-03-25 11:16:19,416 - INFO - Epoch 045: Avg. Loss: 1.35085e-01 Remaining Time: 08h 00min 43s LR: 4.72752e-04 


2025-03-25 11:17:24,000 - LIGHT_DEBUG - Batch 45/80 Pred noise min/max: -0.12196, 0.13229 std/mean: 0.92922, -0.00070 Real noise min/max: -0.12874, 0.13922 std/mean: 1.00000, 0.00005  

: 